In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("OPENWEATHER_API_KEY")
ACCESS_KEY = os.getenv("LAKEFS_ACCESS_KEY")
SECRET_KEY = os.getenv("LAKEFS_SECRET_KEY")
lakefs_endpoint = os.getenv("LAKEFS_ENDPOINT", "http://lakefs-dev:8000")


In [2]:
#test read parquet
import pandas as pd
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

repo = "pollution-data"
branch = "main"
path = "pollution.parquet"
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

In [14]:
path_partition = 's3a://pollution-data/main/pollution.parquet/'
# path_partition = 's3a://pollution-data/main/pollution.parquet/year=2025/month=5/day=11/hour=10/5127aa0fd46841dcba3ad95008af1c7d-0.parquet'
df = pd.read_parquet(
    path=path_partition,
    storage_options=storage_options,
    engine="pyarrow"
    )
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1920 entries, 0 to 1919
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype                       
---  ------            --------------  -----                       
 0   timestamp         1920 non-null   datetime64[ns]              
 1   minute            1920 non-null   int64                       
 2   localtime         1920 non-null   datetime64[ns, Asia/Bangkok]
 3   province          1920 non-null   object                      
 4   district          1900 non-null   object                      
 5   lat               1920 non-null   float64                     
 6   lon               1920 non-null   float64                     
 7   main.aqi          1900 non-null   float64                     
 8   components_co     1900 non-null   float64                     
 9   components_no     1900 non-null   float64                     
 10  components_no2    1900 non-null   float64                     
 11  comp

,timestamp,minute,localtime,province,district,lat,lon,main.aqi,components_co,components_no,components_no2,components_o3,components_so2,components_pm2_5,components_pm10,components_nh3,year,month,day,hour
0,2025-05-11 10:58:23.735003,58,2025-05-11 17:58:23.735003+07:00,Phetchabun,Khao Kho,16.5757,100.9355,2.0,651.68,0.00,10.88,37.37,1.94,23.84,25.46,6.00,2025,5,11,10
1,2025-05-11 10:58:23.734104,58,2025-05-11 17:58:23.734104+07:00,Saraburi,Sao Hai,14.5698,100.8226,1.0,415.57,0.01,10.58,35.68,3.73,9.21,14.10,3.44,2025,5,11,10
2,2025-05-11 10:58:23.735361,58,2025-05-11 17:58:23.735361+07:00,Udon Thani,Prachaksinlapakhom,17.2547,103.0011,2.0,527.83,0.00,5.80,40.10,0.58,14.51,14.99,1.95,2025,5,11,10
3,2025-05-11 10:58:23.735474,58,2025-05-11 17:58:23.735474+07:00,Bangkok,Bangkok Noi,13.7548,100.4694,1.0,171.56,0.09,1.45,14.44,0.39,1.94,2.73,1.14,2025,5,11,10
4,2025-05-11 10:58:23.734865,58,2025-05-11 17:58:23.734865+07:00,Nakhon Pathom,Kamphaeng Saen,14.0341,99.9465,1.0,175.51,0.06,1.06,15.89,0.25,1.54,2.01,1.69,2025,5,11,10


In [ ]:
# lakefs_s3_path,
# storage_options=storage_options,
# partition_cols=["year", "month", "day", "hour"],


In [6]:
import s3fs
path = 's3a://pollution-data/main/pollution.parquet'
fs = s3fs.S3FileSystem(
    key=ACCESS_KEY,
    secret=SECRET_KEY,
    client_kwargs={'endpoint_url': lakefs_endpoint}
)

data_list = fs.glob(f"{path}/*/*/*/*/*")
len(data_list)

27

In [8]:
import pyarrow.dataset as ds
path = "pollution-data/main/pollution.parquet/"
dataset = ds.dataset(
    path,
    format="parquet",
    partitioning=["year", "month", "day", "hour"],
    filesystem=fs
)

table = dataset.to_table()
df = table.to_pandas()
df.info()

FileNotFoundError: pollution-data/main/pollution.parquet/year=2025/month=5/day=13/hour=3/b082cc4d7f844fb2b219eee5018d1fc9-0.parquet

In [15]:
df_filtered = df["hour"] == 14
df_filtered

# filtered = dataset.to_table(filter=(ds.field("hour") == 14))
# df_filtered = filtered.to_pandas()

0       False
1       False
2       False
3       False
4       False
        ...  
1915    False
1916    False
1917    False
1918    False
1919    False
Name: hour, Length: 1920, dtype: bool

In [ ]:
from datetime import datetime

filtered = dataset.to_table(
    filter=(
        (ds.field("year") == 2025) &
        (ds.field("month") == 5) &
        (ds.field("day") == 8) &
        (ds.field("hour") == 14)
    )
)
df_filtered = filtered.to_pandas()


In [9]:
print(df_filtered.head())
print(df_filtered['localtime'].min(), df_filtered['localtime'].max())

NameError: name 'df_filtered' is not defined